<a href="https://colab.research.google.com/github/valerio-unifei/ECAA07/blob/main/ECAA07_PIMS_02_Tags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PIMS 02 - TAGs

## Baixando Arquivos da Aplicação

In [ ]:
!wget https://github.com/valerio-unifei/ECAA07/raw/main/Bancos/ECAA07-PIMS2.zip -O ECAA07-PIMS2.zip
!unzip ECAA07-PIMS2.zip


Estrutura do PIMS
*   templates - pasta com modelos das páginas da aplicação Web
*   PIMS_UNIFEI.py - código fonte com detalhes apresentados na aula passada
*   fabrica1.csv - Arquivo CSV com TAGs a serem importados na aplicação



## Bibliotecas

In [ ]:
!pip install flask flask-sqlalchemy flask-login flask_ngrok

In [3]:
from flask import Flask # classe de aplicação web em flask
from flask import render_template # utiliza modelos em páginas web
from flask import redirect # redireciona o navegador para outra url
from flask import url_for # gera a url de uma rota do flask
from flask import request # coleta as variáveis passadas pelo navegador
from flask import flash # envia um valor de retorno para o conteúdo da página
from werkzeug.security import generate_password_hash # gera um hash criptografado de uma senha
from werkzeug.security import check_password_hash # compara hashs de senha atual e gravada
from flask_sqlalchemy import SQLAlchemy # classe getora de banco de dados 
from flask_login import UserMixin # modelo de usuário para login
from flask_login import LoginManager # classe gerenciadora de login para automação
from flask_login import login_required # indica rotas que necessitam de login para entrar
from flask_login import login_user # grava o usuário que logou na sessão
from flask_login import current_user # obtem usuário logado na sessão
from flask_login import logout_user # apaga usuário logado da sessão
from os import urandom # gerador de código aleatório para chave de segurança
from datetime import datetime # data e hora
from flask_ngrok import run_with_ngrok

## Código da Aula Anterior Melhorado

In [4]:
# cria a aplicação web e pasta com modelos de páginas
app = Flask('PIMS UNIFEI', template_folder='templates')
# chave única do site para assinatura dos cookies
app.config['SECRET_KEY'] = urandom(12)

# gerenciamento de banco de dados
db = SQLAlchemy()
# indica banco de dados da aplicação em sqlite3
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///pims.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False # tira notificações do db
#conecta gerenciador de banco na aplicação
db.init_app(app)

# modelo para objeto usuário no banco
class Usuario(UserMixin, db.Model):
  id = db.Column(db.Integer, primary_key=True)
  usuario = db.Column(db.String(255))
  email = db.Column(db.String(150), unique=True)
  senha = db.Column(db.String(300))
  criado = db.Column(db.DateTime)
  
# gestão de login automatizada
login_manager = LoginManager()
login_manager.login_view = 'login'
login_manager.init_app(app)

@login_manager.user_loader
def load_user(usuario_id):
  # guarda a chave primária do usuário após login
  return Usuario.query.get(int(usuario_id))

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/login')
def login():
    return render_template('login.html')

@app.route('/signup')
def signup():
    return render_template('signup.html')
    
@app.route('/signup', methods=['POST'])
def signup_post():
    email = request.form.get('email')
    usuario = request.form.get('name')
    senha = request.form.get('password')
    # verifica se o usuário já existe no banco
    user = Usuario.query.filter_by(email = email).first() 
    # se existe rediciona a criação para tentar novamente com outro usuário
    if user:
        flash('E-mail de usuário já existe no site')
        return redirect(url_for('signup'))
    # montando dados do usuário para salvar no banco
    new_user = Usuario(
        email = email, 
        usuario = usuario, 
        senha = generate_password_hash(senha, method='sha256'),
        criado = datetime.now(),
        )
    #  inserindo o usuário no banco de dados
    db.session.add(new_user)
    db.session.commit()
    return redirect(url_for('login'))

@app.route('/login', methods=['POST'])
def login_post():
    email = request.form.get('email')
    senha = request.form.get('password')
    remember = True if request.form.get('Lembre-me') else False
    user = Usuario.query.filter_by(email=email).first()
    # se usuário ou hash da senha não são iguais ao do banco
    if not user or not check_password_hash(user.senha, senha):
        flash('Usuário ou senha incorretos')
        return redirect(url_for('login'))
    #cadastra usuário na memória da sessão
    login_user(user, remember=remember)
    # se tudo ok, redireciona a página principal
    return redirect(url_for('profile'))

@app.route('/profile')
@login_required # proteção de entrada com login
def profile():
    # redireciona a página do usuário com 
    return render_template('profile.html', user = current_user)

@app.route('/logout')
@login_required # proteção de entrada com login
def logout():
    # descadastra usuário da memória
    logout_user() 
    # rediciona a página inicial da aplicação
    return redirect(url_for('index'))

app.url_map

## Novas Tabelas do Banco de Dados

In [5]:
# modelo de registro para um tag de entrada ou saída
class Tag(db.Model):
  id = db.Column(db.Integer, primary_key=True)
  nome = db.Column(db.String(50))
  criado = db.Column(db.DateTime)
  ativo = db.Column(db.Boolean)
  medidas = db.relationship('Medida', backref='tag', lazy=True)

# modelo de registro para uma medida realizada no tag
class Medida(db.Model):
  id = db.Column(db.Integer, primary_key=True)
  instante = db.Column(db.DateTime)
  valor = db.Column(db.Float)
  tag_id = db.Column(db.Integer, db.ForeignKey('tag.id'),nullable=False)

# cria banco
with app.app_context():
  db.create_all()

## Rotas para Tabela e Gráficos da Aplicação

In [ ]:
@app.route('/tabela')
@login_required
def tabela():
  tags = Tag.query.all()
  return render_template('tabela.html',tags=tags)

@app.route('/grafico/<tag_id>')
@login_required
def grafico(tag_id):
  
    tag_sel = Tag.query.filter_by(id=tag_id).first()
    medidas = Medida.query.filter_by(tag_id=tag_id).all()

    dados = [m.valor for m in medidas]
    nomes = [m.instante.strftime('%Y-%m-%d %H:%M:%S') for m in medidas]

    return render_template('grafico.html',
      data=dados,
      labels=nomes,
      title=tag_sel.nome,
      )

app.url_map

## Importando Arquivo CSV

In [ ]:
import pandas as pd
# Lendo arquivo CSV
fabrica = pd.read_csv('fabrica1.csv', index_col=0).drop(['machine_status'], axis=1)
#converte coluna texto para data-hora
fabrica[fabrica.columns[0]] = pd.to_datetime(fabrica[fabrica.columns[0]])

fabrica.head()

In [ ]:
#executa no contexto da aplicação
with app.app_context():
  #verifica se não existe registros
  if Tag.query.count() == 0:

    for i in range(1,len(fabrica.columns)):
      nome = fabrica.columns[i]
      print('Importanto TAG:',nome)
      # insere nome das colunas como tags
      ntag = Tag(
        nome = nome,
        criado = datetime.now(),
        ativo = True,
        )
      db.session.add(ntag)
      db.session.commit()

      for k, row in fabrica.iterrows():
        # insere medidas
        med = Medida(
          tag_id = ntag.id, #chave primária do tag
          instante = row['timestamp'],
          valor = row[fabrica.columns[i]],
          )
        db.session.add(med)
    db.session.commit()
  else:
    print('Os tags já foram importados.')

## Executa Aplicação Web

In [ ]:
run_with_ngrok(app)
app.run()